In [ ]:
!pip install beautifulsoup4
!pip install google
!pip install wikipedia-api
!pip install yake
!pip install keybert
!pip install datasets
import numpy as np
from keybert import KeyBERT
import wikipediaapi
from googlesearch import search
from re import finditer
import json
import pandas as pd
import os
import yake
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import collections
import transformers
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
from collections import Counter

keyw_list = ['yake', 'keybert', 'mixed']
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 2 #16
data_collator = DefaultDataCollator(return_tensors="tf")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
h5_path = '/content/drive/MyDrive/tf_model_distilbert_v2.h5'
model_checkpoint = "distilbert-base-uncased"
keyw_extr = keyw_list[2]
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model.load_weights(h5_path)

In [4]:
def extract_keyword(question):
  if keyw_extr == 'yake':
    print('------------')
    kw_extractor = yake.KeywordExtractor()
    keywords_y = kw_extractor.extract_keywords(question)
    keyword_yake = keywords_y[0][0]
    print('NAME TO SEARCH SELECTED:', keyword_yake, '(with YAKE)')
    print('------------')
    return keyword_yake
  elif keyw_extr == 'keybert':
    print('------------')
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    keyword_kbert = kw_model.extract_keywords(question, keyphrase_ngram_range=(1, 3), stop_words='english', highlight=False, top_n=5)[0][0]
    print('NAME TO SEARCH SELECTED:', keyword_kbert, '(with KeyBERT)')
    print('------------')
    return keyword_kbert
  elif keyw_extr == 'mixed':
    #YAKE EXTRACTION
    print('------------')
    kw_extractor = yake.KeywordExtractor()
    keywords_y = kw_extractor.extract_keywords(question)
    keyword_yake = keywords_y[0][0]
    print('NAME TO SEARCH SELECTED:', keyword_yake, '(with YAKE)')
    #KEYBERT EXTRACTION
    print('------------')
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    keyword_kbert = kw_model.extract_keywords(question, keyphrase_ngram_range=(1, 3), stop_words='english', highlight=False, top_n=5)[0][0]
    print('NAME TO SEARCH SELECTED:', keyword_kbert, '(with KeyBERT)')
    print('------------')
    return [keyword_yake, keyword_kbert]

import time

def generate_wikipedia_namepages(search_name):
  namelist = []
  print('started search as: ',str(search_name), 'wikipedia')
  search_urls_it =  search(str(search_name) + 'wikipedia', stop = 5,num=5, lang="en",pause = 3)
  timer0 = time.time()
  search_urls = list(search_urls_it)
  print('list done in: ', time.time()-timer0, ' seconds')
  for url in search_urls:
    if 'https://en.wikipedia.org/wiki/' in url:
      namelist.append(url.replace('https://en.wikipedia.org/wiki/','').replace('_', ' ').replace("%27", "'"))
  return namelist

def get_articles_from_namepage(namepage):
  if  wiki_wiki.page(namepage).exists() == True:
    sezioni = str(wikipediaapi.Wikipedia('en').page(namepage).sections)
    for match in finditer(", Section: See also", str(sezioni)):
      matches = match.span()
    try:
      first_clean = str(sezioni)[0:matches[0]]
    except:
      first_clean = str(sezioni)
    for match in finditer(", Section: External links", str(sezioni)):
      matches2 = match.span()
    try:
      first_clean = str(sezioni)[0:matches2[0]]
    except:
      try:
        first_clean = first_clean
      except:
        first_clean = str(sezioni)
    list_section = []
    for match in finditer("Section:", str(sezioni)):
      list_section.append(match.span()[0])
    list_subsection = []
    for match in finditer("Subsections", str(sezioni)):
      list_subsection.append(match.span()[0])
    splitted_str = []
    for i in range(0, len(list_subsection)):
      splitted_str.append(first_clean[list_section[i]:list_subsection[i]-2])
    cleaned_arr = []
    cleaned_arr.append(wiki_wiki.page(namepage).summary.replace('\n', '. '))
    for e in splitted_str:
      if len(e)>0:
        for match in finditer(":\n", e):
          cleaned_arr.append(e[match.span()[1]::].replace('\n', '. '))
    return list(set(cleaned_arr))
  else:
    return [0]


def collect_articles(extracted_name):
  if keyw_extr == 'mixed' and extracted_name[0] != extracted_name[1]:
    namelist_yake = generate_wikipedia_namepages(extracted_name[0])
    print('Wikipedia pages selected by yake:\n', namelist_yake)
    print('------------')
    namelist_keybert = generate_wikipedia_namepages(extracted_name[1])
    print('Wikipedia pages selected by keybert:\n', namelist_keybert)
    print('------------')
    namelist = list(set([*namelist_yake, *namelist_keybert]))
    print('Wikipedia pages selected by both:\n', namelist)
    print('------------')
    
  else:
    namelist = generate_wikipedia_namepages(extracted_name)
    print('Wikipedia pages selected:\n', namelist)
    print('------------')
  articles_by_names = {}
  for names in namelist:
    articles_by_names[names] = get_articles_from_namepage(names)
  return articles_by_names

wiki_wiki = wikipediaapi.Wikipedia('en')

In [5]:
def similarity_btw_doc(query, context, vectorizer = TfidfVectorizer()):
  vectors = vectorizer.fit_transform([query, context])
  return cosine_similarity(vectors,vectors)[0,-1]

def clean_dict(simil_dict):
  cleaned_dict = {}
  maximum = 0
  for keys in simil_dict:
    key_max = max(simil_dict[keys],key=lambda y: float(y[0]))[0]
    if maximum < key_max:
      maximum = key_max
  
  try:
    tresh = maximum * 0.6
  except:
    tresh = 0
  for keys in simil_dict:
    cleaning_list = simil_dict[keys]
    for elements in sorted(cleaning_list):
      if elements[0] < tresh:
        cleaning_list.remove(elements)
    if len(cleaning_list)>0:
      if cleaning_list[-1][0] < tresh:
        del cleaning_list[-1]
      if len(cleaning_list) != 0:
        cleaned_dict[keys] = cleaning_list 
  ###just for debug
  print('KEYS AND NUMBER OF ELEMENTS TO THE NET\n')
  for key in cleaned_dict.keys():
    print(str(key)+':', len(cleaned_dict[key]))
  ###
  return cleaned_dict

def context_similarity(query, context_dict):
  simil_dict = {}
  for key in context_dict.keys():
    similarities = []
    max = 0
    for i in range(0, len(context_dict[key])):
      if similarity_btw_doc(query,str(context_dict[key][i]))>0:
        similarities.append((similarity_btw_doc(query,str(context_dict[key][i])), i))
    simil_list = sorted(similarities,key=lambda y: float(y[0]),reverse=True)  
    if len(simil_list) > 0 and len(simil_list) < 6:
      simil_dict[key] = simil_list
    elif len(simil_list) >= 6:
      simil_dict[key] = simil_list[0:5]
    
  return clean_dict(simil_dict)

def generate_final_dict(cleaned_dict, context_dict):
  final_dict = {}
  for keys in cleaned_dict.keys():
    article_list = []
    for elements in cleaned_dict[keys]:
      article_list.append(context_dict[keys][elements[1]])
    final_dict[keys] = article_list[::-1]
  return final_dict

In [6]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30
):
    # Build a map example to its corresponding features.
    dict_scores = {}

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    for example_index, example in enumerate(examples):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    try: 
                         
                        start_char = offset_mapping[start_index][0] 
                        end_char = offset_mapping[end_index][1]
                        valid_answers.append( 
                            { 
                                "score": start_logits[start_index] + end_logits[end_index], 
                                "text": context[start_char:end_char], 
                            } 
                        ) 

                    except IndexError: 
                        start_char = 0
                        end_char = 0 
                        valid_answers.append( 
                            { 
                                "score": 0, 
                                "text": context[start_char:end_char], 
                            } 
                        ) 

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
            dict_scores[best_answer['text']] = best_answer['score']

        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer 
        answer = (
            best_answer["text"] if best_answer["score"] > min_null_score else ""
        )
        predictions[example["id"]] = answer

    return predictions, dict_scores

In [7]:
def prepare_question_test(final_d, question):
    
    from datasets import Dataset
    arr_cont = []
    arr_tit = []
    for keys in list(final_d.keys()):
        for contx in list(final_d[keys]):
            arr_cont.append(contx)
            arr_tit.append(keys)

    test_data = {}
    questions = [question] * len(arr_cont)
    names = arr_tit
    ads = list(range(0,len(arr_cont)))
    test_data['context'] = arr_cont
    test_data['id'] = ads
    test_data['title'] = names
    test_data['question'] = questions
    ds = pd.DataFrame(test_data)
    test_ds = Dataset.from_pandas(ds)
    return test_ds

def generate_prediction(test_ds):
    validation_features_test = test_ds.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_ds.column_names,
    )

    validation_dataset_test = validation_features_test.to_tf_dataset(
        columns=["attention_mask", "input_ids"],
        shuffle=False,
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    runtime_prediction_test = model.predict(validation_dataset_test)

    final_predictions_test, dict_scores = postprocess_qa_predictions(
        test_ds,
        validation_features_test,
        runtime_prediction_test["start_logits"],
        runtime_prediction_test["end_logits"],
    )
    
    return final_predictions_test, dict_scores

def voting_by_probability(predictions, probabilities): 
  max_v = max(probabilities.values())
  min_v = min(probabilities.values())
  counter_dict = Counter(predictions)
  norm_dict = {}
  for keys in probabilities.keys():
    norm_dict[keys] = (probabilities[keys] - min_v)/(max_v - min_v)
  plot_results(norm_dict, 'normalized answer confidence score')
  results_dict = {}
  for keys in counter_dict.keys():
      results_dict[keys] = counter_dict[keys] * norm_dict[keys]/2
  try:
    plot_results(results_dict, 'final answer confidence score')
    print("FINAL CANDIDATE'S PROBABILITY", sorted(results_dict.items(), key=lambda x:x[1], reverse=True))
    print('-----------')
    return max(results_dict, key = results_dict.get)
  except:
    return 'NO ANSWER FOUND :('


def ask_me(question):
    name_to_search = extract_keyword(question)
    dict_articles = collect_articles(name_to_search)
    similarity_dict = context_similarity(question, dict_articles)
    final_d = generate_final_dict(similarity_dict, dict_articles)
    ds_question = prepare_question_test(final_d, question)
    if len(ds_question) != 0:
      predictions, probabilities = generate_prediction(ds_question)
      prediction_clean = [i for i in predictions.values() if i]
      print('Asked question: ', question)
      print('Provided answer: ', voting_by_probability(prediction_clean, probabilities))
      return voting_by_probability(prediction_clean, probabilities)
    else:
      print('Asked question: ', question)
      print('Provided answer: ', 'NO ANSWER FOUND :(' )
      return 'NO ANSWER FOUND :('

In [8]:
def plot_results(similarity_dict, title):
  plt.figure(figsize=(8,5))  
  plotdata = pd.DataFrame( 
      {"answer": similarity_dict.values()},  
      index=similarity_dict.keys()) 
  plotdata['answer'].plot(kind="bar", title="test") 
  plt.title(title, fontsize=16)  
  plt.ylabel('confidence score')  
  plt.xlabel('answer')
  plt.close()

In [ ]:
import matplotlib.pyplot as plt
keyw_extr = keyw_list[0]
ask_me("in which city Pietro Mennea was born?")

***Let's consider DBPEDIA to evaluate the accuracy of keyword extractor***

In [ ]:
import pandas as pd
from tqdm import tqdm
df_pedia = pd.read_json('train_dbpedia.json')
objs = [df_pedia, pd.DataFrame(df_pedia['Questions'].tolist()).iloc[:, :3]]
df_pedia = pd.concat(objs, axis=1).drop('Questions', axis=1).drop('DatasetName', axis = 1).drop('ID', axis = 1)
df_pedia['Query'] = df_pedia['Query'].str.lower()
df_pedia['Subject'] = df_pedia['Subject'].str.lower()
df_pedia['Subject'] = df_pedia['Subject'].str.replace('http://dbpedia.org/resource/','')
df_pedia['Subject'] = df_pedia['Subject'].str.replace('_',' ')
for i in tqdm(range(0, df_pedia.shape[0])):
  if df_pedia['Query'][i][-1] == '.':
    df_pedia['Query'][i] = df_pedia['Query'][i][:-1]
  if df_pedia['Subject'][i][-1] == '.':
    df_pedia['Subject'][i] = df_pedia['Subject'][i][:-1]
  if df_pedia['Query'][i][-1] != '?':
    df_pedia['Query'][i] = df_pedia['Query'][i] + '?'
  if df_pedia['Query'][i].find(df_pedia['Subject'][i]) == -1:
    df_pedia['Query'][i] = 'not valid'
df_pedia = df_pedia[df_pedia['Query'] != 'not valid'].reset_index().drop('index', axis = 1)
df_pedia.head()

***Extract the keyword with the chosen model and write them into a txt file***

In [ ]:
create_keywords = False

if create_keywords == True : 
    keywords = []
    for ele in tqdm(df_pedia['Query']):
      try:
        keywords.append(extract_keyword(ele)[0])
      except:
        keywords.append('_')

    with open("keywords_yake.txt", "w") as output:
        output.write(str(keywords))
else:
    text_file = open("keywords_yake.txt", "r")
    keywords = text_file.read().split(',')
    keywords = [s.replace("'", "") for s in keywords]
    keywords = [s.replace("[", "") for s in keywords]

***Metrics for model validation***

In [ ]:
import re
import string
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_metrics(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0, 0, 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)

  return f1, precision, recall

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def iterative_metrics(df,keywords):
  f1s,precisions, recalls = [],[],[]
  for i in range(0, len(keywords)):
    metrics = compute_metrics(df[i], keywords[i])
    try:
      f1s.append(metrics[0])
      precisions.append(metrics[1])
      recalls.append(metrics[2])
    except:
      0
  return f1s, precisions, recalls

***Compute the metrics for keyword extractor***

In [ ]:
f1s, precisions, recalls = iterative_metrics(df_pedia['Query'], keywords)
print(np.mean(f1s), np.mean(precisions), np.mean(recalls))

***Evaluation on answers extracted with the model***

In [ ]:
df_question = pd.read_csv('/content/drive/MyDrive/natural_questions_500.csv')
model_answers = False
answ = []

if model_answers == True: 
  with open('answers_mixed.txt', 'r') as f:
    answ = json.loads(f.read())
else: 
  
  for index,question in enumerate(df_question['Question']):
    answ.append(ask_me(question))
    print('REAL ANSWER:', df_question['Answer'][index])
    with open('/content/drive/MyDrive/answers_mixed.txt', 'w') as f:
      f.write(json.dumps(answ))

------------
NAME TO SEARCH SELECTED: seasonal monsoon winds (with YAKE)
------------
NAME TO SEARCH SELECTED: monsoon winds trade (with KeyBERT)
------------
started search as:  seasonal monsoon winds wikipedia
list done in:  3.593761682510376  seconds
Wikipedia pages selected by yake:
 ['Monsoon', 'Monsoon of South Asia', 'North American monsoon', 'Climate of India', 'East Asian Monsoon']
------------
started search as:  monsoon winds trade wikipedia
list done in:  3.6362524032592773  seconds
Wikipedia pages selected by keybert:
 ['Trade winds', 'Prevailing winds', 'Westerlies', 'Volta do mar', 'Hadley cell']
------------
Wikipedia pages selected by both:
 ['Volta do mar', 'East Asian Monsoon', 'Trade winds', 'Climate of India', 'Hadley cell', 'Prevailing winds', 'Monsoon of South Asia', 'Monsoon', 'North American monsoon', 'Westerlies']
------------
KEYS AND NUMBER OF ELEMENTS TO THE NET

Trade winds: 2
Climate of India: 1
Prevailing winds: 1
Monsoon of South Asia: 5
Monsoon: 2


  0%|          | 0/1 [00:00<?, ?ba/s]

Asked question:  what purpose did seasonal monsoon winds have on trade?
FINAL CANDIDATE'S PROBABILITY [('rain', 0.6828910112380981), ('steering flow', 0.5), ("to cross the world's oceans for centuries", 0.46228718757629395), ('lost their moisture out to the ocean', 0.3852219581604004)]
-----------
Provided answer:  rain
FINAL CANDIDATE'S PROBABILITY [('rain', 0.6828910112380981), ('steering flow', 0.5), ("to cross the world's oceans for centuries", 0.46228718757629395), ('lost their moisture out to the ocean', 0.3852219581604004)]
-----------
REAL ANSWER: enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans
------------
NAME TO SEARCH SELECTED: war and order (with YAKE)
------------
NAME TO SEARCH SELECTED: hp mean war (with KeyBERT)
------------
started search as:  war and order wikipedia
list done in:  3.6817054748535156  seconds
Wikipedia pages selected by yake:
 ['This War of Mine', 'This War of Mine#Gamep

  0%|          | 0/1 [00:00<?, ?ba/s]

Asked question:  what does hp mean in war and order?
Provided answer:  NO ANSWER FOUND :(
REAL ANSWER: hit points or health points
------------
NAME TO SEARCH SELECTED: acute hemolytic reaction (with YAKE)
------------
NAME TO SEARCH SELECTED: rbc acute hemolytic (with KeyBERT)
------------
started search as:  acute hemolytic reaction wikipedia
list done in:  3.7123522758483887  seconds
Wikipedia pages selected by yake:
 ['Acute hemolytic transfusion reaction', 'Acute hemolytic transfusion reaction#Signs and symptoms', 'Acute hemolytic transfusion reaction#Mechanism', 'Acute hemolytic transfusion reaction#Treatment', 'Blood transfusion']
------------
started search as:  rbc acute hemolytic wikipedia
list done in:  3.5629420280456543  seconds
Wikipedia pages selected by keybert:
 ['Hemolytic anemia', 'Hemolysis', 'Hemolysis#Inside the body', 'Hemolysis#Intrinsic causes', 'Hemolysis#Extrinsic causes']
------------
Wikipedia pages selected by both:
 ['Acute hemolytic transfusion reaction#

...

Asked question:  what does the board of directors consist of?
FINAL CANDIDATE'S PROBABILITY [('an executive committee that jointly supervise the activities of an organization', 0.3377322107553482)]
-----------
Provided answer:  an executive committee that jointly supervise the activities of an organization
FINAL CANDIDATE'S PROBABILITY [('an executive committee that jointly supervise the activities of an organization', 0.3377322107553482)]
-----------
REAL ANSWER: a recognized group of people who jointly oversee the activities of an organization
------------
NAME TO SEARCH SELECTED: texas bowl (with YAKE)
------------
NAME TO SEARCH SELECTED: payout texas bowl (with KeyBERT)
------------
started search as:  texas bowl wikipedia
list done in:  3.6504533290863037  seconds
Wikipedia pages selected by yake:
 ['Texas Bowl', 'Texas Bowl#History', 'Texas Bowl#Game results', 'Texas Bowl#MVPs', 'Texas Bowl#Most appearances']
------------
started search as:  payout texas bowl wikipedia
list done

  0%|          | 0/1 [00:00<?, ?ba/s]

Asked question:  what is the payout for the texas bowl?
Provided answer:  NO ANSWER FOUND :(
REAL ANSWER: US$3 million per team
------------
NAME TO SEARCH SELECTED: wine producing states (with YAKE)
------------
NAME TO SEARCH SELECTED: wine producing states (with KeyBERT)
------------
started search as:  ['wine producing states', 'wine producing states'] wikipedia
list done in:  3.7376317977905273  seconds
Wikipedia pages selected:
 ['Wine of the United States', 'List of wine-producing regions', 'History of American wine', 'Wine', 'American Viticultural Area']
------------
KEYS AND NUMBER OF ELEMENTS TO THE NET

Wine of the United States: 5
List of wine-producing regions: 5
History of American wine: 1
Wine: 5
American Viticultural Area: 3


  0%|          | 0/1 [00:00<?, ?ba/s]

Asked question:  what are the top five wine producing states?
FINAL CANDIDATE'S PROBABILITY [('Italy, Spain, France, the United States, and China', 0.5)]
-----------
Provided answer:  Italy, Spain, France, the United States, and China
FINAL CANDIDATE'S PROBABILITY [('Italy, Spain, France, the United States, and China', 0.5)]
-----------
REAL ANSWER: California
------------
NAME TO SEARCH SELECTED: game of thrones (with YAKE)
------------
NAME TO SEARCH SELECTED: season game thrones (with KeyBERT)
------------
started search as:  game of thrones wikipedia
list done in:  3.653897523880005  seconds
Wikipedia pages selected by yake:
 ['Game of Thrones', 'List of Game of Thrones episodes', 'Game of Thrones (season 1)', 'Game of Thrones (season 8)', 'Game of Thrones Theme']
------------
started search as:  season game thrones wikipedia


In [ ]:
f1s, precisions, recalls = iterative_metrics(df_question['Answer'], answ)
print(np.mean(f1s), np.mean(precisions), np.mean(recalls))

In [ ]:
treshold = 0.45
model_sent = SentenceTransformer('bert-base-nli-mean-tokens')
# We compute the embeddings both for ground truth and computed answers
truth_embeddings = model_sent.encode(df_question['Answer'])
answ_embeddings = model_sent.encode(answ)
print(truth_embeddings.shape)
print(answ_embeddings.shape)
def semantic_similarity(truth_embeddings,answ_embeddings, treshold):
  answers_labels = []
  scores = []
  for i in range(0,answ_embeddings.shape[0]):
      sim = cosine_similarity([truth_embeddings[i]], [answ_embeddings[i]] )
      scores.append(sim)
      if sim <= treshold:
        answers_labels.append('Wrong')
      else:
        answers_labels.append('Correct')
  return answers_labels, scores

answers_labels, scores = semantic_similarity(truth_embeddings,answ_embeddings, treshold)